# Endpoints

In [4]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [10]:
df_items = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Data\Procesada\items.parquet')
df_juegos = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Data\Procesada\juegos.parquet')
df_resenias = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Data\Procesada\resenias.parquet')

In [3]:
df_items.sample()

,user_id,items_count,item_id,playtime_forever
739402,76561197970675875,122,346110,644


In [4]:
df_juegos.sample()

,genres,title,price,id,developer,release_year
20095,action,killer instinct,39.99,577940,iron galaxy,2017


In [5]:
df_resenias.sample()

,user_id,item_id,recommend,analisis_sentimiento
43012,comradebeeblebrox,219640,True,1


In [11]:
# Se cambia el nombre de la columna "id" por "item_id" para poder realizar el merge de los dataframes
df_juegos.rename(columns={'id': 'item_id'}, inplace=True)
df_juegos.sample()

,genres,title,price,item_id,developer,release_year
26105,indie,magic matchstick,0.99,718660,anime squad,2017


### Se genran dataframes específicos para cada endpoint.

Endpoint 2.

In [42]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_items_fun_2 = df_items.drop(['playtime_forever'], axis=1)
df_games_fun_2 = df_juegos.drop(['genres','title', 'developer','release_year'], axis=1)
df_reviews_fun_2 = df_resenias.drop(['analisis_sentimiento', 'item_id'], axis=1)

# Se combinan los dataframes.
df_merge = df_reviews_fun_2.merge(df_items_fun_2, on='user_id', how='outer')
df_fun_2 = df_merge.merge(df_games_fun_2, on='item_id', how='outer')

# Se procesa el dataframe para reducir el uso de memoria.
df_fun_2 = df_fun_2.drop_duplicates()
df_fun_2['items_count'] = df_fun_2['items_count'].fillna(0)
df_fun_2['items_count'] = df_fun_2['items_count'].astype(np.int32)
df_fun_2['price'] = df_fun_2['price'].fillna(0)
df_fun_2['price'] = df_fun_2['price'].astype(np.float16)
df_fun_2['item_id'] = df_fun_2['item_id'].fillna(0)
df_fun_2['item_id'] = df_fun_2['item_id'].astype(np.int32)
df_fun_2.reset_index(inplace=True, drop=True)
df_fun_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5627692 entries, 0 to 5627691
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      object 
 1   recommend    object 
 2   items_count  int32  
 3   item_id      int32  
 4   price        float16
dtypes: float16(1), int32(2), object(2)
memory usage: 139.5+ MB


In [41]:
df_fun_2['item_id'].max()

2028850

Endpoint 3.

In [54]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_ju3 = df_juegos[['item_id', 'genres', 'release_year']]
df_it3 = df_items[['item_id', 'user_id','playtime_forever']]

# Se realiza el merge de las columnas extraídas.
df_fun_3 = df_ju3.merge(df_it3, on='item_id', how='outer')

# Se procesa el dataframe para reducir el uso de memoria.
df_fun_3['release_year'] = df_fun_3['release_year'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no se trata de años.
df_fun_3 = df_fun_3[df_fun_3['playtime_forever'] != 0]
df_fun_3['release_year'] = df_fun_3['release_year'].astype(np.int16)
df_fun_3['playtime_forever'] = df_fun_3['playtime_forever'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no son necesarios para la función.
df_fun_3 = df_fun_3[df_fun_3['playtime_forever'] != 0]
df_fun_3['playtime_forever'] = df_fun_3['playtime_forever'].astype(np.int16)

In [55]:
df_fun_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7181485 entries, 67 to 10630683
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   genres            object
 2   release_year      int16 
 3   user_id           object
 4   playtime_forever  int16 
dtypes: int16(2), object(3)
memory usage: 246.6+ MB


Endpoint 4 y 5.

In [37]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_juegos_4_5 = df_juegos[['item_id', 'developer', 'release_year']]
df_resenias_4_5 = df_resenias[['item_id', 'analisis_sentimiento','recommend']]

# Se realiza el merge de las columnas extraídas.
df_fun_4_5 = df_resenias_4_5.merge(df_juegos_4_5, on='item_id', how='outer')
df_fun_4_5.head()

,item_id,analisis_sentimiento,recommend,developer,release_year
0,1250,2.0,True,tripwire interactive,2009.0
1,1250,2.0,True,tripwire interactive,2009.0
2,1250,0.0,True,tripwire interactive,2009.0
3,1250,1.0,True,tripwire interactive,2009.0
4,1250,1.0,True,tripwire interactive,2009.0


In [38]:
df_fun_4_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179188 entries, 0 to 179187
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   item_id               179188 non-null  object 
 1   analisis_sentimiento  130764 non-null  float64
 2   recommend             130764 non-null  object 
 3   developer             169095 non-null  object 
 4   release_year          169095 non-null  float64
dtypes: float64(2), object(3)
memory usage: 6.8+ MB


### **Función 1**<br>
**def developer( desarrollador : str )**: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.<br>


### **Función 2**<br>
**def userdata( User_id : str )**: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.<br>
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [21]:
def userdata(user_id):
    # Si el valor no es una cadena de texto se devuelve un mensaje de error.
    if type(user_id) != str:
        return 'Error: El valor ingresado debe ser una palabra'
    # Se normaliza el valor convirtiéndolo a minúscula.
    usuario = user_id.lower()

    resenias_usuario = df_fun_2[(df_fun_2['user_id'] == user_id) & (df_fun_2['item_id'].isin(df_fun_2['item_id']))]

    gasto = resenias_usuario['price'].sum()

    if resenias_usuario['recommend'].sum() == 0:
        porcentaje_recomendacion = 0
    else:
        porcentaje_recomendacion =  (resenias_usuario['recommend'].sum() / len(resenias_usuario)) * 100

    conteo = df_fun_2[df_fun_2['user_id'] == usuario]
    conteo_items = conteo.shape[0]

    datos_usuario = {
        "User X": user_id,
        "Dinero gastado": f"{gasto:.2f} USD",
        "Porcentaje de recomendación": f'{porcentaje_recomendacion}%',
        "Cantidad de items": conteo_items
        }
    
    return datos_usuario

In [22]:
userdata('76561198326700687')

{'User X': '76561198326700687',
 'Dinero gastado': '375.83 USD',
 'Porcentaje de recomendación': '0%',
 'Cantidad de items': 177}

### **Función 3**<br>
**def UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.<br>
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [52]:
def User_For_Genre(genero):
    # Se convierte la cadena de texto ingresada en minúscula.
    genero_m = genero.lower()
    
    # Se corrobora que el valor ingresado esté dentro del dataframe.
    df_genero = df_fun_3[df_fun_3["genres"] == genero_m]

    # Se genera una sumatoria de las horas jugadas por año.
    df_horas_anuales = df_genero.groupby(["release_year"])["playtime_forever"].sum()
    df_horas_anuales = df_horas_anuales.reset_index()

    # Se hace una sumatoria de las horas respecto al usuario.
    df_horas = df_genero.groupby("user_id")["playtime_forever"].sum()
    
    # Se calcula el usuario con mayor cantidad de horas acumuladas.
    top_horas = df_horas.idxmax()

    df_horas_anuales = df_horas_anuales.rename(columns={"release_year": "Año", "playtime_forever": "Horas"})
    horas_anuales = df_horas_anuales.to_dict(orient="records")

    return {f"Usuario con más horas jugadas para género {genero}": top_horas, "Horas jugadas": horas_anuales}

In [53]:
UserForGenre('action')

{'Usuario con más horas jugadas para género action': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 1983, 'Horas': 3473},
  {'Año': 1984, 'Horas': 384},
  {'Año': 1988, 'Horas': 16001},
  {'Año': 1989, 'Horas': 607},
  {'Año': 1990, 'Horas': 18271},
  {'Año': 1991, 'Horas': 2203},
  {'Año': 1992, 'Horas': 1925},
  {'Año': 1993, 'Horas': 23911},
  {'Año': 1994, 'Horas': 119436},
  {'Año': 1995, 'Horas': 209982},
  {'Año': 1996, 'Horas': 66366},
  {'Año': 1997, 'Horas': 358963},
  {'Año': 1998, 'Horas': 2594564},
  {'Año': 1999, 'Horas': 1975459},
  {'Año': 2000, 'Horas': 5901456},
  {'Año': 2001, 'Horas': 1064461},
  {'Año': 2002, 'Horas': 701780},
  {'Año': 2003, 'Horas': 7146588},
  {'Año': 2004, 'Horas': 51262706},
  {'Año': 2005, 'Horas': 10872722},
  {'Año': 2006, 'Horas': 9497307},
  {'Año': 2007, 'Horas': 16000711},
  {'Año': 2008, 'Horas': 25343018},
  {'Año': 2009, 'Horas': 108841258},
  {'Año': 2010, 'Horas': 114405639},
  {'Año': 2011, 'Horas': 211504009},
  {'Año': 2012, 'Horas'

### **Función 4**<br>
**def best_developer_year( año : int )**: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos).<br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [ ]:
def best_developer_year(anio):

    # Se corrobora que el input sea correcto.
    if anio not in df_fun_4_5['release_year'].unique():
        return f"El año {anio} no existe en los registros."
    
    # Filtrar el dataset para obtener solo las filas correspondientes al año dado.
    juegos_del_año = df_fun_4_5[df_fun_4_5['release_year'] == anio]

    # Se calculan las reseñas por desarrolladora.
    resenias = juegos_del_año.groupby('developer')['recommend'].sum().reset_index()

    # Se ordenan los juegos mejor valorados en orden ascendente.
    desarrolladoras = resenias.sort_values(by='recommend', ascending=False)

    # Se ordenan los primeros tres puestos
    oro = desarrolladoras.iloc[0]['developer']
    plata = desarrolladoras.iloc[1]['developer']
    bronce = desarrolladoras.iloc[2]['developer']

    # Crear la lista de diccionarios con los tres primeros lugares
    top3 = [{"Puesto 1": oro.title()}, {"Puesto 2": plata.title()}, {"Puesto 3": bronce.title()}]
    return top3

In [ ]:
for i in range(2008, 2019):
    print(f'{i} : {best_developer_year(i)}')

2008 : [{'Puesto 1': 'Rockstar New England'}, {'Puesto 2': 'Treyarch'}, {'Puesto 3': 'Cd Projekt Red'}]
2009 : [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Tripwire Interactive'}, {'Puesto 3': 'Dice'}]
2010 : [{'Puesto 1': 'Taleworlds Entertainment'}, {'Puesto 2': 'Obsidian Entertainment'}, {'Puesto 3': 'Avalanche Studios'}]
2011 : [{'Puesto 1': 'Re-Logic'}, {'Puesto 2': 'No More Room In Hell Team'}, {'Puesto 3': 'Valve'}]
2012 : [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Gearbox Software,Aspyr (Mac &Amp; Linux)'}, {'Puesto 3': 'Wild Shadow Studios'}]
2013 : [{'Puesto 1': 'Facepunch Studios'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Digital Extremes'}]
2014 : [{'Puesto 1': 'Edge Of Reality'}, {'Puesto 2': 'Endnight Games Ltd'}, {'Puesto 3': 'Coffee Stain Studios'}]
2015 : [{'Puesto 1': 'Psyonix, Inc.'}, {'Puesto 2': 'Kyle Seeley'}, {'Puesto 3': 'Trion Worlds'}]
2016 : [{'Puesto 1': 'Chucklefish'}, {'Puesto 2': 'Reto-Moto'}, {'Puesto 3': 'Doubledutch Games'}]
2017 : [{'Puesto 1': 'Smartl

### **Función 5** <br>
**def developer_reviews_analysis( desarrolladora : str )**: Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.<br>
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [ ]:
def developer_reviews_analysis(desarrolladora):

    # Si el valor no es una cadena de texto se devuelve un mensaje de error.
    if type(desarrolladora) != str:
        return 'Error: El valor ingresado debe ser una palabra'
    
    #Se convierten a minúscula los valores ingresados para compatibilizar con los valores de la columna 'developer'.
    desarrollador = desarrolladora.lower()

    #Se corrobora que el valor ingresado se encuentre en la columna.
    desarrollador = df_fun_4_5[df_fun_4_5['developer'] == desarrollador]
    
    analisis = desarrollador['analisis_sentimiento']
    # Se hace un conteo de las opiniones.
    opinion = analisis.value_counts()

    return {desarrolladora : list([f'Negative: {(opinion.get(0, 0))}', f'Positive: {(opinion.get(2, 0))}'])}

In [ ]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative: 1058', 'Positive: 8028']}

### Carga de datos.  
Se guardan los datasets específicos de los enpoints en archivos en formato parquet para su posterior uso.

Endpoint 2

In [16]:
table = pa.Table.from_pandas(df_fun_2)
pq.write_table(table, 'df_fun_2.parquet')

Endpoint 3.

In [56]:
table = pa.Table.from_pandas(df_fun_3)
pq.write_table(table, 'df_fun_3.parquet')

Endpoint 4 y 5

In [83]:
table = pa.Table.from_pandas(df_fun_4_5)
pq.write_table(table, 'df_fun_4_5.parquet')